In [1]:
def PolyDiff(u, x, deg = 4, diff = 1, width = 5):

    u = u.flatten()
    x = x.flatten()

    n = len(x)
    du = np.zeros((n - 2*width,diff))

    # Take the derivatives in the center of the domain
    for j in range(width, n-width):

        points = np.arange(j - width, j + width + 1)

        # Fit to a polynomial
        poly = np.polynomial.chebyshev.Chebyshev.fit(x[points],u[points],deg)

        # Take derivatives
        for d in range(1,diff+1):
            du[j-width, d-1] = poly.deriv(m=d)(x[j])

    return du

def PDE_FIND_Burgers(u, dt, dx, diff='poly', offset_x = None, offset_t = None, deg_x = 4, deg_t = 4):
    
    num_samples, m, n = u.shape
    if offset_x == None: offset_x = n//10
    if offset_t == None: offset_t = m//10
    T, X = np.linspace(0,(m-1)*dt,m), np.linspace(0,(n-1)*dx,n)
        
    if diff == 'poly': 
        n2, m2 = n-2*offset_x, m-2*offset_t
        u_ = np.zeros((num_samples,m2,n2))
        ut = np.zeros((num_samples,m2,n2))
        ux = np.zeros((num_samples,m2,n2))
        uxx = np.zeros((num_samples,m2,n2))
        for j in tqdm(range(num_samples)):
            for i in range(n2):
                u_[j, :, i] = u[j,offset_t:-offset_x,i+offset_x]
                ut[j, :, i] = PolyDiff(u[j,:,i+offset_x],T,diff=1,width=offset_t,deg=deg_t)[:,0]
            for i in range(m2):
                Du = PolyDiff(u[j,i+offset_t,:],X,diff=2,width=offset_x,deg=deg_x)
                ux[j, i, :] = Du[:, 0]
                uxx[j, i, :] = Du[:, 1]
        u_t = np.array([U[j] for U in ut for j in range(m2)]).flatten()
        u_x = np.array([U[j] for U in ux for j in range(m2)]).flatten()
        u_xx = np.array([U[j] for U in uxx for j in range(m2)]).flatten()
        
    else:
        M = SPDE(BC = 'P').Parabolic_Matrix(n-1, 1, dx, inverse = False).T
        u_ = np.array([U[j] for U in u for j in range(m-1)])
        u_t = np.array([(U[j+1,:]-U[j,:])/dt for U in u for j in range(m-1)]).flatten()
        u_x = np.array([SPDE().discrete_diff(U, N = n, flatten=True, higher = False)/dx for U in u_]).flatten()
        u_xx = np.dot(u_, M).flatten()
        
    u2_x = u_.flatten()*u_x
    truth = lin_reg(fit_intercept = True).fit(np.array([u_xx, u2_x]).T, u_t)
    
    return truth.coef_[0], truth.coef_[1]